In [4]:
!pip install -U transformers datasets scikit-learn sentencepiece

Defaulting to user installation because normal site-packages is not writeable
  Using cached datasets-2.10.1-py3-none-any.whl (469 kB)
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
  Using cached huggingface_hub-0.13.0-py3-none-any.whl (199 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
     |████████████████████████████████| 77 kB 6.2 MB/s  eta 0:00:01
     |████████████████████████████████| 9.8 MB 22.7 MB/s eta 0:00:01
     |████████████████████████████████| 297 kB 43.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 84.0 MB/s eta 0:00:01
  Using cached transformers-4.26.1-py3-none-any.whl (6.3 MB)
     |████████████████████████████████| 7.6 MB 43.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 43.4 MB/s eta 0:00:01
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
     |████████████████████████████████| 60 kB 12.3 MB/s eta 0:00:01
     |██████████████████████████

In [ ]:
!nvidia-smi

In [4]:
import pandas as pd 

df = pd.read_feather("//media/data/mbti-reddit/preprocessed_df_new.feather") #change this to proper path
#'/content/drive/MyDrive/Colab Notebooks/clickbait_hold_X.csv'
df=df.drop(columns=['authors','subreddit'])

print(df.head())

  labels                                           comments
0   ENTP  I think Si inferior is characterized by discon...
1   ENTP  Berating? Its not like this picture of an age-...
2   ENTP  Without questioning anything youre saying, may...
3   ENTP  It makes sense for a community to value utilit...
4   ENTP  Trying to get laid maybe? Ive done charitable ...


In [5]:
df=df.sample(20000, random_state=1) #random sampling
print(df.head())
df['labels'] = df['labels'].replace(['INTP','ISTP','ENTP','ESTP','INFP','ISFP','ENFP','ESFP', \
                                     'INTJ','ISTJ','ENTJ','ESTJ','INFJ','ISFJ','ENFJ','ESFJ'], \
                                    [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]) 
print(df.head())
df=df.rename(columns={'labels':'labels','comments':'text'})
print(df.head())

        labels                                           comments
686284    INFP  Truth is, there will be always someone who's c...
2023079   INTP  Yeah, he's supposed to carefully plan out his ...
2042692   ISTJ  I was diagnosed before I found Reddit but I sp...
3098768   ENTP  But I’m also asking kung may parusa kay owner ...
1224160   INTP  That is a good start. Pay your debts on time, ...
         labels                                           comments
686284        4  Truth is, there will be always someone who's c...
2023079       0  Yeah, he's supposed to carefully plan out his ...
2042692       9  I was diagnosed before I found Reddit but I sp...
3098768       2  But I’m also asking kung may parusa kay owner ...
1224160       0  That is a good start. Pay your debts on time, ...
         labels                                               text
686284        4  Truth is, there will be always someone who's c...
2023079       0  Yeah, he's supposed to carefully plan out his ...
2

In [6]:
from datasets import Dataset

dataset = Dataset.from_pandas(df)
dataset.shuffle(seed=27)
split_set = dataset.train_test_split(test_size=0.2)

/home/deimann/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
dataset

Dataset({
    features: ['labels', 'text', '__index_level_0__'],
    num_rows: 20000
})

In [6]:
type(split_set)

datasets.dataset_dict.DatasetDict

In [7]:
from transformers import AlbertTokenizer, AlbertModel

tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("albert-base-v2", num_labels=16)



Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.LayerNorm.bias', 'predictions.bias', 'predictions.LayerNorm.weight', 'predictions.decoder.weight', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You sho

In [8]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_dataset = split_set.map(preprocess_function, batched=True)


In [9]:
from transformers import DataCollatorWithPadding
#tokenized_datasets = tokenized_datasets.remove_columns(books_dataset["train"].column_names)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [7]:
print(tokenized_dataset)

DatasetDict({
    train: Dataset({
        features: ['labels', 'text', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    test: Dataset({
        features: ['labels', 'text', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4000
    })
})


In [2]:
CUDA_VISIBLE_DEVICES = 1,2

In [10]:
training_args = TrainingArguments(

    output_dir="/home/deimann/mbti-project",

    learning_rate=2e-5,#2e

    per_device_train_batch_size=16,#16

    per_device_eval_batch_size=16,#16

    num_train_epochs=1,

    weight_decay=0.01,

)

trainer = Trainer(

    model=model,

    args=training_args,

    train_dataset=tokenized_dataset["train"],

    eval_dataset=tokenized_dataset["test"],

    tokenizer=tokenizer,

    data_collator=data_collator,

)

trainer.train()

The following columns in the training set don't have a corresponding argument in `AlbertForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `AlbertForSequenceClassification.forward`,  you can safely ignore this message.
/home/deimann/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/opt/anaconda/envs/transformers/lib/python3.8/site-packages/torch/nn/parallel/data_parallel.py:30: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.


Step,Training Loss


RuntimeError: Caught RuntimeError in replica 1 on device 1.
Original Traceback (most recent call last):
  File "/opt/anaconda/envs/transformers/lib/python3.8/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/opt/anaconda/envs/transformers/lib/python3.8/site-packages/torch/nn/modules/module.py", line 727, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/home/deimann/.local/lib/python3.8/site-packages/transformers/models/albert/modeling_albert.py", line 1076, in forward
    outputs = self.albert(
  File "/opt/anaconda/envs/transformers/lib/python3.8/site-packages/torch/nn/modules/module.py", line 727, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/home/deimann/.local/lib/python3.8/site-packages/transformers/models/albert/modeling_albert.py", line 731, in forward
    encoder_outputs = self.encoder(
  File "/opt/anaconda/envs/transformers/lib/python3.8/site-packages/torch/nn/modules/m

In [ ]:
device = cuda.get_current_device() 
device.reset()

In [10]:
!nvidia-smi

Thu Mar  9 16:46:04 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:1B:00.0 Off |                    0 |
| N/A   62C    P0    98W / 250W |  30091MiB / 32510MiB |     59%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  On   | 00000000:1C:00.0 Off |                  N/A |
| 30%   

AB HIER EINFACH IGNORIEREN!!!
!

In [ ]:
from transformers import TextClassificationPipeline

pipeline = TextClassificationPipeline(model=model,
                                      tokenizer=tokenizer,
                                      device=0)

In [ ]:
model_name_or_path=/content/models/checkpoint-5000

In [10]:
training_args = TrainingArguments(

    output_dir="/content/drive/MyDrive/Colab Notebooks/",

    learning_rate=2e-5,#2e

    per_device_train_batch_size=20,#16

    per_device_eval_batch_size=20,#16

    num_train_epochs=10,

    weight_decay=0.01,

)

trainer = Trainer(

    model=model,

    args=training_args,

    train_dataset=tokenized_dataset["train"],

    eval_dataset=tokenized_dataset["test"],

    tokenizer=tokenizer,

    data_collator=data_collator,

)

trainer.train("/content/drive/MyDrive/Colab Notebooks/checkpoint-4000")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Loading model from /content/drive/MyDrive/Colab Notebooks/checkpoint-4000.
The following columns in the training set don't have a corresponding argument in `AlbertForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `AlbertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training 

0it [00:00, ?it/s]

Step,Training Loss
4500,2.279100
5000,2.241800
5500,2.150100
6000,2.006000
6500,1.899500
7000,1.728800
7500,1.623900
8000,1.543700


Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/checkpoint-4500
Configuration saved in /content/drive/MyDrive/Colab Notebooks/checkpoint-4500/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/checkpoint-4500/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Colab Notebooks/checkpoint-4500/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Colab Notebooks/checkpoint-4500/special_tokens_map.json
Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/checkpoint-5000
Configuration saved in /content/drive/MyDrive/Colab Notebooks/checkpoint-5000/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/checkpoint-5000/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Colab Notebooks/checkpoint-5000/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Colab Notebooks/checkpoint-5000/special_tokens_map.json
Saving model checkpoint to /cont

TrainOutput(global_step=8000, training_loss=0.9670639953613281, metrics={'train_runtime': 3971.1481, 'train_samples_per_second': 40.291, 'train_steps_per_second': 2.015, 'total_flos': 2133879299719680.0, 'train_loss': 0.9670639953613281, 'epoch': 10.0})

In [11]:
training_args = TrainingArguments(

    output_dir="/content/drive/MyDrive/Colab Notebooks/",

    learning_rate=2e-5,#2e

    per_device_train_batch_size=20,#16

    per_device_eval_batch_size=20,#16

    num_train_epochs=15,

    weight_decay=0.01,

)

trainer = Trainer(

    model=model,

    args=training_args,

    train_dataset=tokenized_dataset["train"],

    eval_dataset=tokenized_dataset["test"],

    tokenizer=tokenizer,

    data_collator=data_collator,

)

trainer.train("/content/drive/MyDrive/Colab Notebooks/checkpoint-8000")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Loading model from /content/drive/MyDrive/Colab Notebooks/checkpoint-8000.
The following columns in the training set don't have a corresponding argument in `AlbertForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `AlbertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training 

0it [00:00, ?it/s]

Step,Training Loss
8500,1.532800
9000,1.503300
9500,1.413300
10000,1.280600
10500,1.208100
11000,1.072200
11500,0.994600
12000,0.926500


Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/checkpoint-8500
Configuration saved in /content/drive/MyDrive/Colab Notebooks/checkpoint-8500/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/checkpoint-8500/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Colab Notebooks/checkpoint-8500/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Colab Notebooks/checkpoint-8500/special_tokens_map.json
Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/checkpoint-9000
Configuration saved in /content/drive/MyDrive/Colab Notebooks/checkpoint-9000/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/checkpoint-9000/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Colab Notebooks/checkpoint-9000/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Colab Notebooks/checkpoint-9000/special_tokens_map.json
Saving model checkpoint to /cont

TrainOutput(global_step=12000, training_loss=0.4138126703898112, metrics={'train_runtime': 3970.6209, 'train_samples_per_second': 60.444, 'train_steps_per_second': 3.022, 'total_flos': 3197273209484160.0, 'train_loss': 0.4138126703898112, 'epoch': 15.0})

OutOfMemoryError: CUDA out of memory. Tried to allocate 120.00 MiB (GPU 0; 14.75 GiB total capacity; 12.97 GiB already allocated; 62.81 MiB free; 13.90 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [10]:

model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Colab Notebooks/checkpoint-12000", num_labels=16)

In [11]:
from transformers import TextClassificationPipeline

pipeline = TextClassificationPipeline(model=model,tokenizer=tokenizer)

In [13]:
from sklearn.metrics import f1_score

true_labels = []
predicted_labels = []

for test in split_set["test"]:
  true_labels.append(test["labels"])
  predicted_label = pipeline(test["text"])[0]['label']
  if predicted_label == 'LABEL_0':
    predicted_labels.append(0)
  elif predicted_label == 'LABEL_1':
    predicted_labels.append(1)
  elif predicted_label == 'LABEL_2':
    predicted_labels.append(2)
  elif predicted_label == 'LABEL_3':
    predicted_labels.append(3)
  elif predicted_label == 'LABEL_4':
    predicted_labels.append(4)
  elif predicted_label == 'LABEL_5':
    predicted_labels.append(5)
  elif predicted_label == 'LABEL_6':
    predicted_labels.append(6)
  elif predicted_label == 'LABEL_7':
    predicted_labels.append(7)
  elif predicted_label == 'LABEL_8':
    predicted_labels.append(8)
  elif predicted_label == 'LABEL_9':
    predicted_labels.append(9)
  elif predicted_label == 'LABEL_10':
    predicted_labels.append(10)
  elif predicted_label == 'LABEL_11':
    predicted_labels.append(11)
  elif predicted_label == 'LABEL_12':
    predicted_labels.append(12)
  elif predicted_label == 'LABEL_13':
    predicted_labels.append(13)
  elif predicted_label == 'LABEL_14':
    predicted_labels.append(14)
  elif predicted_label == 'LABEL_15':
    predicted_labels.append(15)
  else:
    print("unexpected label")
    print(predicted_label)
  
  #to show the predicted labels see below
  #print(test["text"])
  #print(test["label"])
  #print()

print(f1_score(true_labels, predicted_labels, average='binary'))

Token indices sequence length is longer than the specified maximum sequence length for this model (1240 > 512). Running this sequence through the model will result in indexing errors


RuntimeError: ignored

In [ ]:
smaller_split = np.sample()

In [20]:
from sklearn.metrics import f1_score

true_labels = []
predicted_labels = []
a = 0
for test in split_set["test"]:
  a+=1
  if a==300:
    break
  true_labels.append(test["labels"])
  predicted_label = pipeline(test["text"])[0]['label']

  if predicted_label == 'LABEL_0':
    predicted_labels.append(0)
  elif predicted_label == 'LABEL_1':
    predicted_labels.append(1)
  elif predicted_label == 'LABEL_2':
    predicted_labels.append(2)
  elif predicted_label == 'LABEL_3':
    predicted_labels.append(3)
  elif predicted_label == 'LABEL_4':
    predicted_labels.append(4)
  elif predicted_label == 'LABEL_5':
    predicted_labels.append(5)
  elif predicted_label == 'LABEL_6':
    predicted_labels.append(6)
  elif predicted_label == 'LABEL_7':
    predicted_labels.append(7)
  elif predicted_label == 'LABEL_8':
    predicted_labels.append(8)
  elif predicted_label == 'LABEL_9':
    predicted_labels.append(9)
  elif predicted_label == 'LABEL_10':
    predicted_labels.append(10)
  elif predicted_label == 'LABEL_11':
    predicted_labels.append(11)
  elif predicted_label == 'LABEL_12':
    predicted_labels.append(12)
  elif predicted_label == 'LABEL_13':
    predicted_labels.append(13)
  elif predicted_label == 'LABEL_14':
    predicted_labels.append(14)
  elif predicted_label == 'LABEL_15':
    predicted_labels.append(15)
  else:
    print("unexpected label")
    print(predicted_label)
  
  #to show the predicted labels see below
  #print(test["text"])
  #print(test["labels"])
  #print('predicted: '+pipeline(test["text"])[0]['label'])
  #print()

#print(f1_score(true_labels, predicted_labels, average=None))

In [ ]:

### Confusion Matrix
from sklearn.metrics import confusion_matrix
predictions = model.predict(x_test, steps=len(x_test), verbose=0)
#y_pred=model.predict(x_test)
#y_pred = np.round(y_pred)
y_pred = np.argmax(predictions, axis=-1)

y_true=np.argmax(y_test, axis=-1)

cm = confusion_matrix(y_true, y_pred)

## Get Class Labels
labels = le.classes_
class_names = labels

# Plot confusion matrix in a beautiful manner
fig = plt.figure(figsize=(16, 14))
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, fmt = 'g'); #annot=True to annotate cells
# labels, title and ticks
ax.set_xlabel('Predicted', fontsize=20)
ax.xaxis.set_label_position('bottom')
plt.xticks(rotation=90)
ax.xaxis.set_ticklabels(class_names, fontsize = 10)
ax.xaxis.tick_bottom()

ax.set_ylabel('True', fontsize=20)
ax.yaxis.set_ticklabels(class_names, fontsize = 10)
plt.yticks(rotation=0)

plt.title('Refined Confusion Matrix', fontsize=20)

plt.savefig('ConMat24.png')
plt.show()

In [ ]:


df['labels'] = df['labels'].replace(['INTP','ISTP','ENTP','ESTP','INFP','ISFP','ENFP','ESFP', \
                                     'INTJ','ISTJ','ENTJ','ESTJ','INFJ','ISFJ','ENFJ','ESFJ'], \
                                    [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]) 

In [23]:
evaldf = pd.DataFrame(
    {'true': true_labels,
     'predicted': predicted_labels
    })
evaldf['true'] = evaldf['true'].replace([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15], \
                                         ['INTP','ISTP','ENTP','ESTP','INFP','ISFP','ENFP','ESFP', \
                                     'INTJ','ISTJ','ENTJ','ESTJ','INFJ','ISFJ','ENFJ','ESFJ'])
                                    
evaldf['predicted'] = evaldf['predicted'].replace([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15], \
                                         ['INTP','ISTP','ENTP','ESTP','INFP','ISFP','ENFP','ESFP', \
                                     'INTJ','ISTJ','ENTJ','ESTJ','INFJ','ISFJ','ENFJ','ESFJ'])

print(evaldf)


     true predicted
0    ENTP      INTJ
1    INFP      ENFP
2    ESTP      INFP
3    ISFP      ISTJ
4    INFJ      INFP
..    ...       ...
294  ISTP      INTJ
295  INFJ      INTP
296  INTP      INTP
297  INTP      INFP
298  INTJ      ENFP

[299 rows x 2 columns]


In [1]:
print(evaldf[0])

NameError: ignored

In [26]:
evaldf.to_csv(r'/content/drive/MyDrive/Colab Notebooks/evaldf1.csv', index=False)
